In [1]:
!sudo add-apt-repository -y ppa:potassco/stable
!sudo apt-get update
!apt install clingo
!pip install potassco

Repository: 'deb https://ppa.launchpadcontent.net/potassco/stable/ubuntu/ jammy main'
Description:
Contains the stable releases of systems from the https://potassco.org project.
More info: https://launchpad.net/~potassco/+archive/ubuntu/stable
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/potassco-ubuntu-stable-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/potassco-ubuntu-stable-jammy.list
Adding key to /etc/apt/trusted.gpg.d/potassco-ubuntu-stable.gpg with fingerprint 7AA3F20F5BDCBF2F78090C08DE1AB6C94EFE9A64
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/r

In [2]:
!clingo --version

clingo version 5.7.1 (849f282)
Address model: 64-bit

libclingo version 5.7.1
Configuration: with Python 3.10.12, with Lua 5.3.6

libclasp version 3.3.10 (libpotassco version 1.1.0)
Configuration: WITH_THREADS=1
Copyright (C) Benjamin Kaufmann

License: The MIT License <https://opensource.org/licenses/MIT>


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import ast
import heapq
import networkx as nx
import subprocess
import time

In [5]:
csv_path = "https://raw.githubusercontent.com/DigitalSuYee/GPS-Data/main/bus_plan%20(1).csv"
bus_plan = pd.read_csv(csv_path)

bus_plan

,sid1,sid2,route,mins
0,5188,3818,a,2.000000
1,3818,3820,a,5.000000
2,3820,3821,a,8.000000
3,3821,3528,a,5.000000
4,3528,3529,a,2.000000
...,...,...,...,...
415,4099,4006,w,5.537429
416,4216,4291,w,4.408196
417,4291,4216,w,4.408196
418,4218,4222,w,9.236181


In [6]:
bus_plan['route'].unique()

array(['a', 'd', 'c', 'b', 'e', 'w'], dtype=object)

In [7]:
stops_list = sorted(set(bus_plan['sid1']).union(set(bus_plan['sid2'])))
stops_code = " ".join([f"stop(s{i})." for i in stops_list])

In [8]:
stops_code

'stop(s367). stop(s369). stop(s373). stop(s381). stop(s836). stop(s839). stop(s843). stop(s846). stop(s847). stop(s878). stop(s882). stop(s887). stop(s890). stop(s904). stop(s906). stop(s910). stop(s965). stop(s966). stop(s968). stop(s970). stop(s974). stop(s975). stop(s978). stop(s979). stop(s980). stop(s981). stop(s982). stop(s983). stop(s985). stop(s987). stop(s1049). stop(s1050). stop(s1051). stop(s1052). stop(s1054). stop(s1055). stop(s1335). stop(s1339). stop(s1341). stop(s1343). stop(s1346). stop(s1349). stop(s1353). stop(s1355). stop(s1424). stop(s1595). stop(s1598). stop(s1600). stop(s1602). stop(s1606). stop(s1608). stop(s1610). stop(s1612). stop(s1613). stop(s1615). stop(s1619). stop(s1622). stop(s1624). stop(s1629). stop(s1631). stop(s1634). stop(s1642). stop(s1646). stop(s1671). stop(s1676). stop(s1783). stop(s1785). stop(s1786). stop(s1788). stop(s1789). stop(s1791). stop(s1792). stop(s1795). stop(s1820). stop(s1823). stop(s1838). stop(s1887). stop(s1906). stop(s2022). st

In [9]:
lines_code = ""
for index, row in bus_plan.iterrows():
    sid1 = int(row['sid1'])
    sid2 = int(row['sid2'])
    route = str(row['route'])
    mins = int(row['mins'])
    lines_code += f"line(s{sid1}, s{sid2}, {route}, {mins}). "


In [10]:
lines_code

'line(s5188, s3818, a, 2). line(s3818, s3820, a, 5). line(s3820, s3821, a, 8). line(s3821, s3528, a, 5). line(s3528, s3529, a, 2). line(s3529, s381, a, 9). line(s381, s3530, a, 6). line(s3530, s3531, a, 3). line(s3531, s2387, a, 3). line(s2387, s2663, a, 5). line(s2663, s3533, a, 9). line(s3533, s3535, a, 6). line(s3535, s3536, a, 9). line(s3536, s3537, a, 7). line(s3537, s3539, a, 10). line(s3539, s3540, a, 10). line(s3540, s3541, a, 2). line(s3541, s3543, a, 9). line(s3543, s3545, a, 2). line(s3545, s3546, a, 3). line(s3546, s3549, a, 2). line(s3549, s3550, a, 9). line(s3550, s3551, a, 8). line(s3551, s3553, a, 10). line(s3553, s3555, a, 7). line(s3555, s4246, a, 6). line(s4246, s3556, a, 10). line(s3556, s3558, a, 5). line(s3558, s3559, a, 2). line(s3559, s3562, a, 3). line(s3562, s2763, a, 6). line(s2763, s2843, a, 5). line(s2843, s2845, a, 6). line(s2845, s2847, a, 16). line(s2847, s2848, a, 7). line(s2848, s2932, a, 19). line(s2932, s5307, a, 4). line(s5307, s2612, a, 9). line(s2

In [11]:
code = """
line(s5188, s3818, a, 2). line(s3818, s3820, a, 5). line(s3820, s3821, a, 8). line(s3821, s3528, a, 5). line(s3528, s3529, a, 2). line(s3529, s381, a, 9). line(s381, s3530, a, 6). line(s3530, s3531, a, 3). line(s3531, s2387, a, 3). line(s2387, s2663, a, 5). line(s2663, s3533, a, 9). line(s3533, s3535, a, 6). line(s3535, s3536, a, 9). line(s3536, s3537, a, 7). line(s3537, s3539, a, 10). line(s3539, s3540, a, 10). line(s3540, s3541, a, 2). line(s3541, s3543, a, 9). line(s3543, s3545, a, 2). line(s3545, s3546, a, 3). line(s3546, s3549, a, 2). line(s3549, s3550, a, 9). line(s3550, s3551, a, 8). line(s3551, s3553, a, 10). line(s3553, s3555, a, 7). line(s3555, s4246, a, 6). line(s4246, s3556, a, 10). line(s3556, s3558, a, 5). line(s3558, s3559, a, 2). line(s3559, s3562, a, 3). line(s3562, s2763, a, 6). line(s2763, s2843, a, 5). line(s2843, s2845, a, 6). line(s2845, s2847, a, 16). line(s2847, s2848, a, 7). line(s2848, s2932, a, 19). line(s2932, s5307, a, 4). line(s5307, s2612, a, 9). line(s2612, s2614, a, 10). line(s2614, s2618, a, 7). line(s2618, s2622, a, 17). line(s2622, s2850, a, 12). line(s2850, s2851, a, 3). line(s2851, s2854, a, 12). line(s2854, s5308, a, 6). line(s5308, s2855, a, 9). line(s2855, s2859, a, 16). line(s2859, s5309, a, 10). line(s5309, s4774, a, 5). line(s4774, s5310, a, 16). line(s5310, s2862, a, 8). line(s2862, s4776, a, 10). line(s4776, s3767, a, 3). line(s3767, s3768, a, 8). line(s3722, s3723, d, 3). line(s3723, s3726, d, 8). line(s3726, s3728, d, 10). line(s3728, s3729, d, 7). line(s3729, s2898, d, 8). line(s2898, s2899, d, 5). line(s2899, s2900, d, 6). line(s2900, s1791, d, 16). line(s1791, s1792, d, 6). line(s1792, s1795, d, 6). line(s1795, s1823, d, 8). line(s1823, s1820, d, 10). line(s1820, s1823, d, 4). line(s1823, s1783, d, 5). line(s1783, s1785, d, 4). line(s1785, s1786, d, 9). line(s1786, s4827, d, 3). line(s4827, s2905, d, 2). line(s2905, s2906, d, 8). line(s2906, s2909, d, 7). line(s2909, s1838, d, 5). line(s1838, s3750, d, 4). line(s3750, s3746, d, 9). line(s3746, s3750, d, 3). line(s3750, s3753, d, 4). line(s3753, s3757, d, 8). line(s978, s980, c, 8). line(s980, s981, c, 9). line(s981, s3200, c, 7). line(s3200, s3241, c, 3). line(s3241, s3243, c, 6). line(s3243, s4232, c, 4). line(s4232, s3244, c, 10). line(s3244, s3246, c, 4). line(s3246, s982, c, 10). line(s982, s983, c, 6). line(s983, s985, c, 7). line(s985, s987, c, 5). line(s987, s1676, c, 5). line(s1676, s3256, c, 6). line(s4008, s4012, c, 6). line(s4012, s5426, c, 10). line(s5426, s1335, c, 10). line(s1335, s1339, c, 8). line(s1339, s1341, c, 2). line(s1341, s1343, c, 5). line(s1343, s3360, c, 3). line(s3360, s3362, c, 4). line(s3362, s3364, c, 7). line(s3364, s4222, c, 5). line(s4222, s3413, c, 2). line(s3413, s3416, c, 5). line(s3416, s3417, c, 7). line(s3417, s3419, c, 8). line(s3419, s3420, c, 10). line(s3420, s3032, c, 5). line(s3032, s3033, c, 9). line(s3033, s2717, c, 6). line(s2717, s3069, c, 2). line(s3069, s3070, c, 10). line(s3070, s4291, c, 3). line(s4291, s4567, c, 4). line(s4567, s3586, c, 5). line(s3586, s1906, c, 2). line(s1906, s4963, c, 6). line(s3583, s3585, c, 7). line(s3585, s4567, c, 9). line(s4567, s4216, c, 4). line(s4216, s3071, c, 7). line(s3071, s3072, c, 8). line(s3072, s2712, c, 4). line(s2712, s2713, c, 3). line(s2713, s4155, c, 5). line(s4155, s3442, c, 6). line(s3442, s3444, c, 3). line(s3444, s3447, c, 6). line(s3447, s3450, c, 7). line(s3450, s4218, c, 9). line(s4218, s3368, c, 6). line(s3368, s3370, c, 6). line(s3370, s4219, c, 7). line(s4219, s4220, c, 8). line(s4220, s1346, c, 8). line(s1346, s1349, c, 8). line(s1349, s1353, c, 9). line(s1353, s1355, c, 9). line(s1355, s4090, c, 2). line(s4090, s4091, c, 4). line(s4091, s4098, c, 10). line(s4098, s4253, c, 10). line(s4253, s3184, c, 3). line(s3184, s1642, c, 9). line(s1642, s1646, c, 10). line(s1646, s965, c, 8). line(s965, s966, c, 10). line(s966, s968, c, 10). line(s968, s970, c, 4). line(s970, s3192, c, 6). line(s3192, s3195, c, 6). line(s3195, s3199, c, 10). line(s3199, s974, c, 10). line(s974, s979, c, 9). line(s979, s975, c, 2). line(s4768, s3431, b, 10). line(s3431, s1424, b, 2). line(s1424, s3579, b, 4). line(s3579, s4360, b, 15). line(s4360, s2039, b, 9). line(s2039, s2044, b, 2). line(s2044, s2138, b, 12). line(s2138, s2140, b, 10). line(s2140, s1051, b, 6). line(s1051, s1054, b, 2). line(s1054, s1055, b, 16). line(s1055, s3036, b, 5). line(s3036, s3037, b, 5). line(s3037, s4822, b, 6). line(s4822, s3983, b, 12). line(s3983, s3984, b, 11). line(s3984, s5348, b, 3). line(s5348, s3989, b, 14). line(s3989, s3992, b, 11). line(s3992, s3821, b, 9). line(s3821, s381, b, 19). line(s381, s4099, b, 12). line(s4099, s4101, b, 10). line(s4101, s4004, b, 5). line(s4004, s3076, b, 8). line(s3076, s373, b, 7). line(s373, s3083, b, 9). line(s3083, s3084, b, 7). line(s3084, s3091, b, 6). line(s3091, s3095, b, 10). line(s3095, s3097, b, 6). line(s3097, s1671, b, 7). line(s1671, s2031, b, 5). line(s2031, s4488, b, 3). line(s4488, s2476, b, 8). line(s2476, s2477, b, 6). line(s2477, s2022, b, 9). line(s2022, s2024, b, 8). line(s2024, s1887, b, 8). line(s1887, s3044, b, 5). line(s3044, s3047, b, 4). line(s3047, s3050, b, 6). line(s3050, s3053, b, 4). line(s3053, s3056, b, 6). line(s3056, s3061, b, 8). line(s3061, s3066, b, 4). line(s3066, s4006, b, 4). line(s4006, s3560, b, 2). line(s3560, s3566, b, 6). line(s3566, s3568, b, 7). line(s3568, s3571, b, 5). line(s3571, s3573, b, 5). line(s3573, s5401, b, 3). line(s5401, s2223, b, 4). line(s2223, s2228, b, 3). line(s2228, s2231, b, 4). line(s2231, s2232, b, 5). line(s2232, s4257, b, 9). line(s4257, s2237, b, 4). line(s2237, s2240, b, 5). line(s2240, s3812, b, 10). line(s3812, s3814, b, 9). line(s3814, s3917, b, 5). line(s3917, s3920, b, 4). line(s3920, s3925, b, 2). line(s3925, s3926, b, 9). line(s3926, s4157, b, 9). line(s4157, s367, b, 5). line(s367, s3927, b, 7). line(s3927, s1788, b, 6). line(s1788, s1789, b, 4). line(s1789, s1049, b, 8). line(s1049, s1050, b, 5). line(s1050, s1052, b, 10). line(s1052, s2133, b, 5). line(s2133, s2135, b, 4). line(s2135, s2038, b, 4). line(s2038, s2040, b, 6). line(s2040, s4768, b, 4). line(s878, s5477, e, 7). line(s5477, s836, e, 7). line(s836, s839, e, 10). line(s839, s882, e, 7). line(s882, s887, e, 15). line(s887, s890, e, 18). line(s890, s1595, e, 15). line(s1595, s1598, e, 18). line(s1598, s1600, e, 12). line(s1600, s1602, e, 9). line(s1602, s1606, e, 20). line(s1606, s1608, e, 6). line(s1608, s1619, e, 8). line(s1619, s1612, e, 6). line(s1619, s3232, e, 22). line(s3232, s3234, e, 13). line(s3234, s3235, e, 6). line(s3235, s1613, e, 10). line(s1613, s3238, e, 15). line(s3238, s3240, e, 14). line(s3240, s5325, e, 6). line(s5325, s3241, e, 8). line(s3241, s3242, e, 8). line(s3242, s4232, e, 6). line(s4232, s3246, e, 19). line(s3246, s3247, e, 6). line(s3247, s3250, e, 7). line(s3250, s3251, e, 9). line(s3251, s3253, e, 10). line(s3253, s3255, e, 9). line(s3255, s3256, e, 7). line(s3256, s4008, e, 5). line(s4008, s4010, e, 6). line(s4010, s4012, e, 4). line(s5426, s4014, e, 3). line(s4014, s4016, e, 6). line(s4016, s4017, e, 5). line(s4017, s5212, e, 4). line(s5212, s5213, e, 6). line(s5213, s2381, e, 3). line(s2381, s5214, e, 9). line(s5214, s5215, e, 9). line(s5215, s2382, e, 9). line(s2382, s2384, e, 9). line(s2384, s2386, e, 3). line(s4086, s4088, e, 10). line(s4088, s4090, e, 8). line(s4090, s4092, e, 8). line(s4092, s4094, e, 6). line(s4094, s4097, e, 10). line(s4097, s4253, e, 14). line(s3184, s3185, e, 2). line(s3185, s3189, e, 9). line(s3189, s3190, e, 7). line(s3190, s3191, e, 10). line(s3191, s3192, e, 6). line(s3192, s3194, e, 10). line(s3194, s3197, e, 9). line(s3197, s3199, e, 12). line(s3199, s3200, e, 2). line(s3200, s3201, e, 10). line(s3201, s3205, e, 12). line(s3205, s1615, e, 18). line(s1615, s3209, e, 15). line(s3209, s3210, e, 12). line(s3210, s3230, e, 12). line(s3230, s1610, e, 13). line(s1610, s1622, e, 23). line(s1622, s1624, e, 18). line(s1624, s1629, e, 22). line(s1629, s1631, e, 6). line(s1631, s1634, e, 12). line(s1634, s904, e, 30). line(s904, s906, e, 33). line(s906, s910, e, 35). line(s910, s843, e, 19). line(s843, s847, e, 8). line(s847, s846, e, 9). line(s367, s3983, w, 4). line(s3983, s367, w, 4). line(s369, s5426, w, 7). line(s5426, s369, w, 7). line(s373, s3061, w, 9). line(s3061, s373, w, 9). line(s381, s2237, w, 4). line(s2237, s381, w, 4). line(s878, s5477, w, 2). line(s5477, s878, w, 2). line(s887, s906, w, 9). line(s906, s887, w, 9). line(s965, s987, w, 9). line(s987, s965, w, 9). line(s974, s979, w, 1). line(s979, s974, w, 1). line(s1049, s1055, w, 6). line(s1055, s1049, w, 6). line(s1050, s1054, w, 3). line(s1054, s1050, w, 3). line(s1051, s1052, w, 5). line(s1052, s1051, w, 5). line(s1595, s1634, w, 7). line(s1634, s1595, w, 7). line(s1598, s1631, w, 6). line(s1631, s1598, w, 6). line(s1600, s1629, w, 7). line(s1629, s1600, w, 7). line(s1610, s1619, w, 8). line(s1619, s1610, w, 8). line(s1788, s3037, w, 7). line(s3037, s1788, w, 7). line(s1789, s3036, w, 6). line(s3036, s1789, w, 6). line(s1791, s4827, w, 6). line(s4827, s1791, w, 6). line(s2038, s2044, w, 6). line(s2044, s2038, w, 6). line(s2133, s2140, w, 3). line(s2140, s2133, w, 3). line(s2135, s2138, w, 7). line(s2138, s2135, w, 7). line(s2223, s3543, w, 6). line(s3543, s2223, w, 6). line(s2228, s3536, w, 7). line(s3536, s2228, w, 7). line(s2231, s3533, w, 5). line(s3533, s2231, w, 5). line(s2384, s5213, w, 8). line(s5213, s2384, w, 8). line(s2387, s2663, w, 6). line(s2663, s2387, w, 6). line(s3050, s3091, w, 7). line(s3091, s3050, w, 7). line(s3056, s3084, w, 9). line(s3084, s3056, w, 9). line(s3066, s3076, w, 5). line(s3076, s3066, w, 5). line(s3069, s3072, w, 6). line(s3072, s3069, w, 6). line(s3070, s3071, w, 5). line(s3071, s3070, w, 5). line(s3189, s3251, w, 4). line(s3251, s3189, w, 4). line(s3190, s3250, w, 5). line(s3250, s3190, w, 5). line(s3192, s3246, w, 5). line(s3246, s3192, w, 5). line(s3194, s3244, w, 6). line(s3244, s3194, w, 6). line(s3195, s4232, w, 7). line(s4232, s3195, w, 7). line(s3197, s3242, w, 5). line(s3242, s3197, w, 5). line(s3199, s3241, w, 5). line(s3241, s3199, w, 5). line(s3209, s3234, w, 7). line(s3234, s3209, w, 7). line(s3210, s3232, w, 6). line(s3232, s3210, w, 6). line(s3362, s3370, w, 9). line(s3370, s3362, w, 9). line(s3413, s3450, w, 3). line(s3450, s3413, w, 3). line(s3416, s3447, w, 5). line(s3447, s3416, w, 5). line(s3420, s3442, w, 7). line(s3442, s3420, w, 7). line(s3546, s5401, w, 8). line(s5401, s3546, w, 8). line(s3549, s3573, w, 7). line(s3573, s3549, w, 7). line(s3559, s3562, w, 6). line(s3562, s3559, w, 6). line(s3917, s3992, w, 4). line(s3992, s3917, w, 4). line(s3920, s3989, w, 9). line(s3989, s3920, w, 9). line(s3926, s3984, w, 9). line(s3984, s3926, w, 9). line(s4006, s4099, w, 5). line(s4099, s4006, w, 5). line(s4216, s4291, w, 4). line(s4291, s4216, w, 4). line(s4218, s4222, w, 9). line(s4222, s4218, w, 9).


start(s2899).
end(s1791).

0 { selected(X,Y,Z) } 1 :- line(X,Y,Z,_).

path(X,Y) :- selected(X,Y,_).
path(X,Z) :- path(X,Y), path(Y,Z).
path(X, Z, W) :- path(X, Y, W), path(Y, Z, W).

:- start(X), end(Y), not path(X,Y).

cost(C) :- C = #sum { W,X,Y,Z : line(X,Y,Z,W), selected(X,Y,Z) }.

#minimize { C : cost(C) }.

#show selected/3.
"""

with open('program4.lp', 'w') as file:
    file.write(code)

In [12]:
!clingo program4.lp

clingo version 5.7.1 (849f282)
Reading from program4.lp
Solving...
Answer: 1
selected(s3818,s3820,a) selected(s3820,s3821,a) selected(s3821,s3528,a) selected(s3528,s3529,a) selected(s381,s3530,a) selected(s3531,s2387,a) selected(s2663,s3533,a) selected(s3536,s3537,a) selected(s3537,s3539,a) selected(s3541,s3543,a) selected(s3545,s3546,a) selected(s3546,s3549,a) selected(s3549,s3550,a) selected(s3550,s3551,a) selected(s3551,s3553,a) selected(s3559,s3562,a) selected(s3562,s2763,a) selected(s2763,s2843,a) selected(s2843,s2845,a) selected(s2845,s2847,a) selected(s2847,s2848,a) selected(s2848,s2932,a) selected(s2932,s5307,a) selected(s5307,s2612,a) selected(s2612,s2614,a) selected(s3722,s3723,d) selected(s3723,s3726,d) selected(s3726,s3728,d) selected(s3728,s3729,d) selected(s3729,s2898,d) selected(s2898,s2899,d) selected(s2899,s2900,d) selected(s2900,s1791,d) selected(s1791,s1792,d) selected(s1792,s1795,d) selected(s1795,s1823,d) selected(s1823,s1820,d) selected(s1823,s1783,d) selected(s17